<a href="https://colab.research.google.com/github/ariefbillah91/UAS-NLP/blob/main/UAS_NLP_MetodeTwitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp 'kaggle.json' ~/.kaggle
!chmod 600 ~/.kaggle/'kaggle.json'
!kaggle datasets download -d 'jp797498e/twitter-entity-sentiment-analysis'

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [20]:
import zipfile
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

In [21]:
def get_one_hot_encoded(label):
  if label.lower() == 'positive':
    return [1, 0, 0]
  elif label.lower() == 'negative':
    return [0, 1, 0]
  else:
    return [0, 0, 1]

In [22]:
def remove_url (text):
  # http://google.com adalah bla bla bla
  # https://youtube.com
  return re.sub(r'http\S+', '', text)

def remove_number(text):
  # 123 saya blablabla
  return re.sub(r'\d+', '', text)

def remove_stopwords(text):
  words = text.split() # saya mau ke batam -> [saya, mau, ke, batam]
  new_text = ""
  for word in words:
    if word not in stopwords.words('english'):
      new_text += word
      new_text += " "

  return new_text

def lematized_text(text):
  lematizer = WordNetLemmatizer()
  words = text.split()
  new_text = ""
  for word in words:
    lematized_word = lematizer.lemmatize(word)
    new_text += lematized_word
    new_text += " "
  return new_text
  def stemmed_text(text):
  stemmer = PorterStemmer()
  words = text.split()
  new_text = ""
  for word in words:
    stemmed_word = stemmer.stem(word)
    new_text += stemmed_word
    new_text += " "
  return new_text

def preprocessing (text):
  #lowercase -> Running, running, RUNNING, RunNinG
  text = text.lower()

  # remove url
  text = remove_url(text)

  # remove number
  text = remove_number(text)

  # remove stopwords
  text = remove_stopwords(text)

  # lematisasi
  text = lematized_text(text)

  # stemming
  text = stemmed_text(text)

  return text

IndentationError: expected an indented block after function definition on line 29 (<ipython-input-22-f2ad0fa9bccb>, line 30)

In [23]:
# "ini bapak budi" -> 3 kata
# "ini ibu budi" -> 3 kata
# "saya dan bapak ibu budi pergi ke pasar" -> 8 kata
# 0, 0, 0 , 0, 0

def get_avg (text_list):
  sum = 0
  for text in text_list:
    word_num = len(text)
    sum = sum + word_num
  return sum/len(text_list) # total kata di dataset / banyak kalimat yang ada

In [24]:
import pickle

def tokenizing(text_list):
  VOCAB_SIZE = 10000
  tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token='')
  tokenizer.fit_on_texts(text_list)
  word_index = tokenizer.word_index
  max_length = get_avg(text_list)
  print(max_length)
  max_length = int(max_length)
  print(max_length)
  sequences = tokenizer.texts_to_sequences(text_list)
  padded_sequences = pad_sequences(sequences, padding='pre', truncating='pre', maxlen=max_length)

  with open('mytokenizer.pickle', 'wb') as file:
    pickle.dump(tokenizer, file, protocol=pickle.HIGHEST_PROTOCOL)

  return padded_sequences

In [25]:
labels = np.array(labels)

NameError: name 'labels' is not defined

In [26]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=10001, output_dim=100, input_length=57),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100, activation = 'relu', input_shape=(None, 75)),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [27]:
model.compile(
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

In [28]:
model.save('mymodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
